<a href="https://colab.research.google.com/github/ARAN-ai-python/UsedcarValuePrediction_AI/blob/main/jupyter%20notebook/1.%E4%B8%AD%E5%8F%A4%E8%BB%8A%E4%BE%A1%E6%A0%BC%E4%BA%88%E6%B8%ACAI(%E4%BE%A1%E6%A0%BC_com)_%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Car_Bigdata

In [ ]:
from bs4 import BeautifulSoup
import requests
from time import sleep
import json
import pandas as pd
from tqdm.notebook import tqdm


pd.set_option('display.max_columns',None)


#pref[1~55] page[1~60]
#長丁場になりそう
page_start = 1
page_end = 60
car_start = 1
car_end = 20
pref_start = 1
pref_end = 55
pref_code = 55
url = "https://kakaku.com/kuruma/used/spec/Prefecture={}/Page={}/"

In [ ]:
pre_results = []

In [ ]:
#価格.comから中古車データを都道府県毎にスクレイピングするプログラム
#Used_Car_Data_Scraping
def UCDS(start,end,pref_code,pre_results=[]):
    #pre_resultsに値があった場合(再開する場合)、読み込んだページ数をprogressに記録
    d_list = pre_results
    if len(d_list) > 0:
        progress = list(pd.DataFrame(d_list)['ページ'].unique())
    else:
        progress = []
        
    print("Scraping Start!")

    #tqdmで進捗を監視
    for i in tqdm(range(start, end+1)):

        try:
            target_url = url.format(pref_code, i)
            print(target_url)
            r = requests.get(target_url)
            sleep(1)
            soupr = BeautifulSoup(r.content, 'html.parser')
            contents = soupr.find_all('div', class_="itemImgBox u-pRight15")
            #print(contents)
            name_index = 0
            for content in tqdm(contents):
                car_links = content.find_all("a")
                #print(car_links)
                names = soupr.find_all('a', onclick="if('function' == typeof window.onclickcatalyst) onclickcatalyst('spec_text');")
                #print(names[name_index].text)
                for car_link in car_links:
                    #既に読み込んだページは飛ばして効率化を図る
                    if (car_link.get('href')[18:-1]) in progress:
                        continue
                    
                    l = requests.get("https://kakaku.com"+car_link.get('href'))
                    sleep(1)
                    soupl = BeautifulSoup(l.content, 'html.parser')
                    info1 = soupl.find("table", class_="specList")
                    info2 = soupl.find("div", class_="carinfoArea")
                    #print(info1)

                    page = car_link.get('href')[18:-1]
                    #print(page)

                    name = names[name_index].text
                    name_index += 1
                    print(name)
                    value = soupl.find_all("span", class_="num")[1].text
                    #print(value)
                    total = soupl.find("span", class_="total").text
                    #print(total)
                    color = info1.find_all('td')[0].text
                    #print(color)
                    year = info1.find_all('td')[1].text
                    #print(year)
                    distance = info1.find_all('td')[2].text
                    #print(distance)
                    area = info1.find_all('td')[3].text
                    #print(area)
                    inspection = info1.find_all('td')[4].text
                    #print(inspection)
                    repair = info1.find_all('td')[5].text
                    #print(repair)
                    unused = info1.find_all('td')[6].text
                    #print(unused)
                    oneowner = info1.find_all('td')[7].text
                    #print(oneowner)
                    nosmoking = info1.find_all('td')[8].text
                    #print(nosmoking)
                    guarantee = info1.find_all('td')[9].text
                    #print(guarantee)

                    #print("\n")

                    mission = info2.find_all('dd')[0].text
                    #print(mission)
                    drive = info2.find_all('dd')[3].text
                    #print(drive)
                    displacement = info2.find_all('dd')[6].text
                    #print(displacement)
                    door = info2.find_all('dd')[9].text
                    #print(door)
                    capacity = info2.find_all('dd')[12].text
                    #print(capacity)
                    fuel = info2.find_all('dd')[15].text
                    #print(fuel)
                    handle = info2.find_all('dd')[1].text
                    #print(handle)
                    number = info2.find_all('dd')[4].text
                    #print(number)
                    record = info2.find_all('dd')[7].text
                    #print(record)
                    recycle = info2.find_all('dd')[10].text
                    #print(recycle)
                    maintenance = info2.find_all('dd')[13].text
                    #print(maintenance)
                    judgement = info2.find_all('dd')[16].text
                    #print(judgement)
                    length = info2.find_all('dd')[2].text
                    #print(length)
                    width = info2.find_all('dd')[5].text
                    #print(width)
                    height = info2.find_all('dd')[8].text
                    #print(height)
                    weight = info2.find_all('dd')[11].text
                    #print(weight)
                    fuelconsumption = info2.find_all('dd')[14].text
                    #print(fuelconsumption)
                    usingfuel = info2.find_all('dd')[17].text
                    #print(usingfuel)

                    #print("\n")
                    
                    options = soupl.find_all('li', class_='yes')
                    #print(options)
                    options_list = ""
                    for option in options:
                        #print(option.text)
                        options_list += str(option.text) + " "
                    #print(options_list)

                    d = {
                        "ページ":page
                        ,"車名":name

                        #info1
                        ,"価格":value
                        ,"総額":total
                        ,"色":color
                        ,"年式":year
                        ,"走行距離":distance
                        ,"地域":area
                        ,"車検":inspection
                        ,"修復歴":repair
                        ,"未使用車":unused
                        ,"ワンオーナー":oneowner
                        ,"禁煙車":nosmoking
                        ,"保証":guarantee

                        #info2
                        ,"ミッション":mission
                        ,"駆動方式":drive
                        ,"排気量":displacement
                        ,"ドア数":door
                        ,"乗車定員":capacity
                        ,"燃料":fuel
                        ,"ハンドル":handle
                        ,"車台番号":number
                        ,"記録簿":record
                        ,"リサイクル料":recycle
                        ,"整備":maintenance
                        ,"ID車両(グー鑑定)":judgement
                        ,"全長":length
                        ,"全幅":width
                        ,"全高":height
                        ,"車両重量":weight
                        ,"燃費":fuelconsumption
                        ,"使用燃料":usingfuel
                        
                        #info3
                        ,"オプション":options_list
                    }

                    d_list.append(d)

        #リストにある存在しないページにアクセスした場合、そのページの読み込みをスキップする。
        except IndexError:
            continue

        #何らかの理由で中断された場合、その時点までに読み込んだデータを保存する。
        except:
            break

    print("Scraping Completed!")
    return d_list


In [ ]:
results = UCDS(page_start, page_end, pref_code, pre_results)
pre_results = results
display(pd.DataFrame(pre_results))

Scraping Start!


  0%|          | 0/60 [00:00<?, ?it/s]

https://kakaku.com/kuruma/used/spec/Prefecture=55/Page=1/


  0%|          | 0/20 [00:00<?, ?it/s]

ムーヴ Ｌ
ムーヴ カスタム
ハイゼットカーゴ ＤＸ
キャスト スタイルＸ
キャスト スタイルＧ
キャスト スタイルＸ
ハイゼットカーゴ ＤＸ
キャスト スタイルＸ
キャスト スタイルＸ
キャスト スタイルＧ
ムーヴ カスタム
ムーヴ カスタム
キャスト スタイルＧ
ミラトコット Ｌ
ｅＫクロススペース Ｍ
ミライース Ｘ
タント カスタムＸ
ミライース Ｘ
ミライース Ｘ
ミライース Ｘ
https://kakaku.com/kuruma/used/spec/Prefecture=55/Page=2/


...

  0%|          | 0/20 [00:00<?, ?it/s]

キャスト スタイルＸ
カローラアクシオ ハイブリッドＧ
タント Ｘ
ＢＲＺ ＳＴＩ
ＭＡＺＤＡ２ １５Ｓプロアクティブ
シフォン カスタムＲＳ
ミライース Ｌ
ウェイク ＧターボリミテッドＳＡＩＩＩ
Ｎ−ＢＯＸ Ｇ・Ｌホンダセンシング
カローラフィールダー １．５Ｇ
キャスト アクティバＸ
デリカＤ：５ Ｇ
エスクァイア Ｇｉ
カローラフィールダー １．５Ｇ
ムーヴ Ｌ
Ｎ−ＢＯＸ Ｇ・Ｌホンダセンシング
フィットハイブリッド Ｌ
クラウンハイブリッド Ｇ
シエンタ Ｇ
アルトラパン モード
https://kakaku.com/kuruma/used/spec/Prefecture=55/Page=60/


  0%|          | 0/20 [00:00<?, ?it/s]

アクア Ｓスタイルブラック
フレア ６６０
ＳＸ４ Ｓクロス ベースグレード
アルト Ｌ
デイズ ６６０
ノート １．２
Ｃ−ＨＲ ハイブリッドＧ
Ａクラスセダン Ａ２５０
カローラフィールダー ハイブリッド
ルーミー カスタムＧ
ワゴンＲ ２５周年記念車
ｅＫワゴン Ｍ
ｅＫスペース Ｇ
トール カスタムＧ
ハイゼットカーゴ クルーズＳＡＩＩＩ
シエンタ ハイブリッドＸ
クラウンハイブリッド Ｓ
ｅＫワゴン Ｍ
カローラフィールダー １．５Ｇ
プレミオ １．５Ｆ
Scraping Completed!


,ページ,車名,価格,総額,色,年式,走行距離,地域,車検,修復歴,未使用車,ワンオーナー,禁煙車,保証,ミッション,駆動方式,排気量,ドア数,乗車定員,燃料,ハンドル,車台番号,記録簿,リサイクル料,整備,ID車両(グー鑑定),全長,全幅,全高,車両重量,燃費,使用燃料,オプション
0,26634376,ムーヴ Ｌ,98.0,（総額 120.0万円）,ブライトシルバーＭ,令和3年(2021年),2km,沖縄県,2024/01,修復歴なし,−,○,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,CVT,2WD,660cc,5ドア,4名,ガソリン,右ハンドル,702（下3桁）,,リ済別,法定整備付,,"3,395mm※1","1,475mm※1","1,630mm※1",820kg※1,26.4km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ キーレスエントリー エアバッグ ABS
1,26634372,ムーヴ カスタム,133.0,（総額 154.0万円）,レーザーブルークリスタルシャイン,令和3年(2021年),2km,沖縄県,2024/01,修復歴なし,−,○,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,CVT,2WD,660cc,5ドア,4名,ガソリン,右ハンドル,726（下3桁）,,リ済別,法定整備付,,"3,395mm※1","1,475mm※1","1,630mm※1",830kg※1,26.4km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ キーレスエントリー エアバッグ ABS アル...
2,27096292,ハイゼットカーゴ ＤＸ,109.0,,ホワイト,令和3年(2021年),3km,沖縄県,2023/03,修復歴なし,−,○,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,MT,2WD,660cc,5ドア,4名,ガソリン,右ハンドル,422（下3桁）,,リ済別,法定整備付,,,,,,,,エアコン パワーステアリング パワーウィンドウ エアバッグ ABS
3,27042636,キャスト スタイルＸ,127.0,,ダークエメラルドマイカ,令和3年(2021年),3km,沖縄県,2024/03,修復歴なし,−,○,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,CVT,2WD,660cc,5ドア,4名,ガソリン,右ハンドル,509（下3桁）,,リ済別,法定整備付,,"3,395mm※1","1,475mm※1","1,600mm※1",840kg※1,25.0km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ キーレスエントリー エアバッグ ABS
4,26985867,キャスト スタイルＧ,137.0,（総額 146.0万円）,シャイニングホワイトパール,令和3年(2021年),3km,沖縄県,2024/03,修復歴なし,○,○,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,CVT,2WD,660cc,5ドア,4名,ガソリン,右ハンドル,578（下3桁）,,リ済別,法定整備付,,"3,395mm※1","1,475mm※1","1,600mm※1",840kg※1,30.0km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ キーレスエントリー エアバッグ ABS アル...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,26873346,シエンタ ハイブリッドＸ,215.0,,パールマイカ,令和元年(2019年),1.0万km,沖縄県,車検整備なし,修復歴なし,−,−,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,AT,2WD,1500cc,5ドア,7名,その他,右ハンドル,057（下3桁）,記録簿あり,リ済別,法定整備付,,"4,235mm※1","1,695mm※1","1,675mm※1","1,380kg※1",27.2km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ バックモニターカメラ テレビ ETC エアバ...
1196,26829511,クラウンハイブリッド Ｓ,397.0,（総額 398.0万円）,パールホワイト,令和元年(2019年),1.0万km,沖縄県,2022/08,修復歴あり,−,−,○,販売店保証付(無料保証 期間:24ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,CVT,2WD,2500cc,4ドア,5名,その他,右ハンドル,702（下3桁）,,リ済込,法定整備付,,"4,910mm※1","1,800mm※1","1,455mm※1","1,740kg※1",24.0km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ カーナビ バックモニターカメラ テレビ CD...
1197,26817739,ｅＫワゴン Ｍ,84.0,,スターリングシルバーメタリック,令和元年(2019年),1.0万km,沖縄県,2022/01,修復歴なし,−,−,−,"販売店保証付(無料保証 期間:24ヵ月 距離:30,000km ※いずれかの条件に達するまで...",CVT,2WD,660cc,5ドア,4名,ガソリン,右ハンドル,117（下3桁）,,リ済別,法定整備なし,,"3,395mm※1","1,475mm※1","1,640mm※1",830kg※1,29.4km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ キーレスエントリー エアバッグ ABS ベン...
1198,25282778,カローラフィールダー １．５Ｇ,198.0,（総額 217.8万円）,パールマイカ,令和元年(2019年),1.0万km,沖縄県,車検整備なし,修復歴なし,−,−,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,AT,2WD,1500cc,5ドア,5名,ガソリン,右ハンドル,773（下3桁）,記録簿あり,リ済別,法定整備付,,"4,410mm※1","1,695mm※1","1,510mm※1","1,140kg※1",23.0km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ エアバッグ ABS アルミホイール


In [ ]:
d_df = pd.DataFrame(results)
for name in list(d_df.columns):
    print(d_df[name].unique())
    print(d_df[name].nunique())

['26634376' '26634372' '27096292' ... '26817739' '25282778' '26078315']
1200
['ムーヴ Ｌ' 'ムーヴ カスタム' 'ハイゼットカーゴ ＤＸ' 'キャスト スタイルＸ' 'キャスト スタイルＧ' 'ミラトコット Ｌ'
 'ｅＫクロススペース Ｍ' 'ミライース Ｘ' 'タント カスタムＸ' 'ハイゼットカーゴ クルーズＳＡＩＩＩ'
 'ハイゼットカーゴ デラックスＳＡＩＩＩ' 'ハスラー Ｊスタイル' 'ハイゼットカーゴ ＤＸエアコンパワステ' 'ウェイク ＧターボＳＡＩＩＩ'
 'ハイゼットトラック ハイルーフエアコンパワステ５速４ＷＤ' 'ミラトコット Ｇ' 'ミラトコット Ｘ'
 'ハイゼットトラック ハイルーフエアコンパワステ５速' 'ｅＫワゴン Ｇ' 'ｅＫワゴン Ｊｏｙ' 'ｅＫスペース Ｇ'
 'ハイゼットカーゴ スペシャル' 'タフト Ｇ' 'タント カスタムＲＳスタイルセレクション' 'ムーヴキャンバス Ｇ' 'ウェイク Ｌ'
 'タント Ｘ' 'タント カスタムＸスタイルセレクション' 'ミライース Ｂ' 'キャリイトラック ＫＣエアコン・パワステ農繁仕様'
 'ミライース Ｇ' 'タント カスタムＸセレクション' 'ワゴンＲ ハイブリッドＦＺ' 'Ｎ−ＢＯＸカスタム Ｌ' 'タント Ｌ'
 'ハイゼットトラック' 'ワゴンＲスティングレー ハイブリッドＸ' 'ウェイク ＬリミテッドＳＡＩＩＩ' 'ハイゼットトラック ハイルーフ'
 'ハイゼットトラック ジャンボ' 'ミライース Ｌ' 'Ｎ−ＷＧＮ Ｌホンダセンシング' 'ハイゼットトラック スタンダード'
 'デリカＤ：５ Ｇ' 'Ｎ−ＢＯＸカスタム Ｌターボ' 'ムーヴキャンバス Ｇブラックインテリアリミテッド'
 'キャリイトラック ＫＣエアコン・パワステ' 'ルークス ６６０' 'ソリオ ハイブリッドＭＺ' 'タント Ｘスペシャル' 'ノート Ｓ'
 'ジムニーシエラ ＪＣ' 'ｅＫクロス Ｔ' 'アルファード ２．５Ｓ' 'ヤリスクロス Ｚ' 'スペーシア ハイブリッドＸ'
 'ステップワゴンスパーダ スパーダ' 'ハイゼットトラック エクストラＳＡＩＩＩｔ' 'エブリイ ＰＣ'
 'ハイゼットカーゴ クルー

In [ ]:
d_df[d_df["オプション"].str.contains('福祉車両')]

,ページ,車名,価格,総額,色,年式,走行距離,地域,車検,修復歴,未使用車,ワンオーナー,禁煙車,保証,ミッション,駆動方式,排気量,ドア数,乗車定員,燃料,ハンドル,車台番号,記録簿,リサイクル料,整備,ID車両(グー鑑定),全長,全幅,全高,車両重量,燃費,使用燃料,オプション
325,26774915,スペーシア 福祉車両,応談,,パールホワイト,令和3年(2021年),228km,沖縄県,2023/03,修復歴なし,−,−,−,"販売店保証付(無料保証 期限:2026年3月 距離:100,000km ※いずれかの条件に達...",CVT,2WD,660cc,5ドア,4名,その他,右ハンドル,976（下3桁）,,リ済込,法定整備付,,,,,,,,エアコン パワーステアリング パワーウィンドウ カーナビ バックモニターカメラ テレビ CD...
345,23579057,Ｎ−ＢＯＸ,157.5,,モーニングミストブルーメタリック,令和2年(2020年),4km,沖縄県,2023/04,修復歴なし,−,−,−,"販売店保証付(無料保証 期間:60ヵ月 距離:100,000km ※いずれかの条件に達するま...",CVT,2WD,660cc,5ドア,,ガソリン,右ハンドル,001（下3桁）,,リ未,法定整備付,,,,,,,,エアコン パワーステアリング パワーウィンドウ バックモニターカメラ CDデッキ ETC キ...
968,24658320,ヴォクシー ＺＳ,応談,,パールホワイト,令和2年(2020年),0.6万km,沖縄県,2022/02,修復歴なし,−,−,−,販売店保証付(無料保証 期間:24ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,CVT,2WD,2000cc,5ドア,7名,ガソリン,右ハンドル,971（下3桁）,記録簿あり,リ済込,法定整備付,,"4,710mm※1","1,730mm※1","1,825mm※1","1,600kg※1",16.0km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ キーレスエントリー エアバッグ ABS 盗難...
1018,26826551,シエンタ Ｘ,205.0,（総額 213.0万円）,ホワイト,令和元年(2019年),885km,沖縄県,車検整備なし,修復歴なし,−,−,−,"販売店保証付(無料保証 期間:24ヵ月 距離:20,000km ※いずれかの条件に達するまで...",CVT,2WD,1500cc,5ドア,5名,ガソリン,右ハンドル,209（下3桁）,,リ済込,法定整備付,外装：5 内装：5 機関：正常 修復歴：無（鑑定日 2021-07-29）,"4,260mm※1","1,695mm※1","1,675mm※1","1,320kg※1",20.2km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ カーナビ バックモニターカメラ キーレスエン...
1051,26140718,ステップワゴン １．５,249.8,（総額 257.6万円）,プラチナホワイトパール,令和元年(2019年),0.4万km,沖縄県,2022/01,修復歴なし,−,−,−,販売店保証付(無料保証 期間:24ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,CVT,2WD,1500cc,5ドア,7名,ガソリン,右ハンドル,131（下3桁）,,リ済別,法定整備付,,,,,,,,エアコン パワーステアリング パワーウィンドウ カーナビ バックモニターカメラ テレビ CD...


In [ ]:
d_df.to_pickle('Car_{}.pickle'.format(pref_code))

In [ ]:
display(pd.read_pickle(f'Car_{pref_code}.pickle'))

,ページ,車名,価格,総額,色,年式,走行距離,地域,車検,修復歴,未使用車,ワンオーナー,禁煙車,保証,ミッション,駆動方式,排気量,ドア数,乗車定員,燃料,ハンドル,車台番号,記録簿,リサイクル料,整備,ID車両(グー鑑定),全長,全幅,全高,車両重量,燃費,使用燃料,オプション
0,26634376,ムーヴ Ｌ,98.0,（総額 120.0万円）,ブライトシルバーＭ,令和3年(2021年),2km,沖縄県,2024/01,修復歴なし,−,○,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,CVT,2WD,660cc,5ドア,4名,ガソリン,右ハンドル,702（下3桁）,,リ済別,法定整備付,,"3,395mm※1","1,475mm※1","1,630mm※1",820kg※1,26.4km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ キーレスエントリー エアバッグ ABS
1,26634372,ムーヴ カスタム,133.0,（総額 154.0万円）,レーザーブルークリスタルシャイン,令和3年(2021年),2km,沖縄県,2024/01,修復歴なし,−,○,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,CVT,2WD,660cc,5ドア,4名,ガソリン,右ハンドル,726（下3桁）,,リ済別,法定整備付,,"3,395mm※1","1,475mm※1","1,630mm※1",830kg※1,26.4km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ キーレスエントリー エアバッグ ABS アル...
2,27096292,ハイゼットカーゴ ＤＸ,109.0,,ホワイト,令和3年(2021年),3km,沖縄県,2023/03,修復歴なし,−,○,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,MT,2WD,660cc,5ドア,4名,ガソリン,右ハンドル,422（下3桁）,,リ済別,法定整備付,,,,,,,,エアコン パワーステアリング パワーウィンドウ エアバッグ ABS
3,27042636,キャスト スタイルＸ,127.0,,ダークエメラルドマイカ,令和3年(2021年),3km,沖縄県,2024/03,修復歴なし,−,○,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,CVT,2WD,660cc,5ドア,4名,ガソリン,右ハンドル,509（下3桁）,,リ済別,法定整備付,,"3,395mm※1","1,475mm※1","1,600mm※1",840kg※1,25.0km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ キーレスエントリー エアバッグ ABS
4,26985867,キャスト スタイルＧ,137.0,（総額 146.0万円）,シャイニングホワイトパール,令和3年(2021年),3km,沖縄県,2024/03,修復歴なし,○,○,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,CVT,2WD,660cc,5ドア,4名,ガソリン,右ハンドル,578（下3桁）,,リ済別,法定整備付,,"3,395mm※1","1,475mm※1","1,600mm※1",840kg※1,30.0km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ キーレスエントリー エアバッグ ABS アル...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,26873346,シエンタ ハイブリッドＸ,215.0,,パールマイカ,令和元年(2019年),1.0万km,沖縄県,車検整備なし,修復歴なし,−,−,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,AT,2WD,1500cc,5ドア,7名,その他,右ハンドル,057（下3桁）,記録簿あり,リ済別,法定整備付,,"4,235mm※1","1,695mm※1","1,675mm※1","1,380kg※1",27.2km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ バックモニターカメラ テレビ ETC エアバ...
1196,26829511,クラウンハイブリッド Ｓ,397.0,（総額 398.0万円）,パールホワイト,令和元年(2019年),1.0万km,沖縄県,2022/08,修復歴あり,−,−,○,販売店保証付(無料保証 期間:24ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,CVT,2WD,2500cc,4ドア,5名,その他,右ハンドル,702（下3桁）,,リ済込,法定整備付,,"4,910mm※1","1,800mm※1","1,455mm※1","1,740kg※1",24.0km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ カーナビ バックモニターカメラ テレビ CD...
1197,26817739,ｅＫワゴン Ｍ,84.0,,スターリングシルバーメタリック,令和元年(2019年),1.0万km,沖縄県,2022/01,修復歴なし,−,−,−,"販売店保証付(無料保証 期間:24ヵ月 距離:30,000km ※いずれかの条件に達するまで...",CVT,2WD,660cc,5ドア,4名,ガソリン,右ハンドル,117（下3桁）,,リ済別,法定整備なし,,"3,395mm※1","1,475mm※1","1,640mm※1",830kg※1,29.4km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ キーレスエントリー エアバッグ ABS ベン...
1198,25282778,カローラフィールダー １．５Ｇ,198.0,（総額 217.8万円）,パールマイカ,令和元年(2019年),1.0万km,沖縄県,車検整備なし,修復歴なし,−,−,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,AT,2WD,1500cc,5ドア,5名,ガソリン,右ハンドル,773（下3桁）,記録簿あり,リ済別,法定整備付,,"4,410mm※1","1,695mm※1","1,510mm※1","1,140kg※1",23.0km/L※1,レギュラー※1,エアコン パワーステアリング パワーウィンドウ エアバッグ ABS アルミホイール


In [ ]:
#スクレイピングメモ(オプションまで終わった都道府県)
#1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
#31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55
#Scraping Complited!

In [ ]:
#urlエンコード　テスト
#テスラ = %83e%83X%83%89

import urllib.parse
tesla = "テスラ"
tesla_quote = urllib.parse.quote(tesla, encoding='shift-jis')
print(tesla_quote)
#これ応用したら任意の車のデータが自動でスクレイピング出来るようになるのでは

%83e%83X%83%89
